In [1]:
import pandas as pd
import os
import glob
import numpy as np
import seaborn as sns

In [2]:
#states DTSC tracker
path = r"C:\Users\jacque.trahan\Downloads\Fires2021_ParcelLayer_v2_InternalView_12.16.21*"
file_type = '.xlsx'
loadup = glob.glob(path + file_type)
df = pd.read_excel(max(loadup, key=os.path.getctime), sheet_name=1)

In [3]:
# df.info(verbose=True)

# Navada

In [4]:
navada = df[(df['County'].isin(['NEVADA']) & (df['Parcel Status:'].isin(['Assessed', 'Deferred to Phase 2',
                                                                             'Identified', 'Assessed Return Needed'])))]

In [5]:
navada = navada.copy()

In [6]:
navada['APN'] = navada['APN'].astype(str)

In [7]:
navada['APN'] = (navada['APN'].str[:-3])

# placer

In [8]:
placer = df[(df['County'].isin(['PLACER']) & (df['Parcel Status:'].isin(['Assessed', 'Deferred to Phase 2',
                                                                             'Identified', 'Assessed Return Needed'])))]

In [9]:
placer =placer.copy()

In [10]:
placer['APN'] = placer['APN'].astype(str)

In [11]:
placer['APN'] = placer['APN'].str.replace("-", "")

In [12]:
placer['APN'] = placer['APN'].str[:-3]

# Lake

In [13]:
lake = df[(df['County'].isin(['LAKE']) & (df['Parcel Status:'].isin(['Assessed', 'Deferred to Phase 2',
                                                                             'Identified', 'Assessed Return Needed'])))]

In [14]:
lake = lake.copy()

In [15]:
lake['APN'] = lake['APN'].astype(str)

In [16]:
lake['APN'] = (lake['APN'] + "0")

# EL DORADO

In [17]:
eldorado = df[(df['County'].isin(['EL DORADO']) & (df['Parcel Status:'].isin(['Assessed', 'Deferred to Phase 2',
                                                                             'Identified', 'Assessed Return Needed'])))]

In [18]:
eldorado = eldorado.copy()

In [19]:
eldorado['APN'] = eldorado['APN'].astype(str)

In [20]:
# eldorado['APN'] = "0" + eldorado['APN']

# concat all the counties into one file

In [21]:
DTSC = pd.concat([navada,placer,lake, eldorado])

In [22]:
# pd.set_option('display.max_rows', 400)

In [23]:
# DTSC['APN'].value_counts()

### Add the dashes

In [24]:
DTSC['APN'] = (DTSC['APN'].str[:3] + '-' + DTSC['APN'].str[3:6] + '-' + DTSC['APN'].str[6:])

In [28]:
DTSC = DTSC[['APN', 'County', 'Parcel Status:', 'Address']]

In [29]:
DTSC_copy = DTSC.copy()
DTSC_copy = DTSC_copy.add_suffix('_DTSC')

# For Randy - Needs Site Assessment Filter

In [30]:
# path3 = r"C:\Users\jacque.trahan\Downloads\Randy filter*"
# file_type3 = '.xlsx'
# load_it3 = glob.glob(path3 + file_type3)
# df3 = pd.read_excel(max(load_it3, key=os.path.getctime))

In [31]:
# Randy_filt = df3.copy()

In [32]:
# Randy_filt = Randy_filt[['APN_ROW Segment', 'ROE Verified','ROE Type', 'Hold',
#                          'Not Eligible','Withdrawal','Phase 1 Complete','Site Assessment', 'USFS Parcel?']]

In [33]:
# Randy_filt = Randy_filt.add_suffix('_Randy')

In [34]:
# DTSC_Randy = DTSC_copy.merge(Randy_filt,
#                                            left_on='APN_DTSC',
#                                            right_on='APN_ROW Segment_Randy',
#                                            how='outer')

# Planning Ready for site assessment tracker. 

## filters for ROEs Available for SA Teams
* ROE Verified = True
* Holds = False
* Not eilibale and widrawn = blank
* phase one = blank
* Site Assessmet = blank
* USFS Parcel? = N/A

In [35]:
#states DTSC tracker
path2 = r"C:\Users\jacque.trahan\Downloads\Central Division 2021 Debris Removal Tracker*"
file_type2 = '.xlsx'
loadup2 = glob.glob(path2 + file_type2)
df2 = pd.read_excel(max(loadup2, key=os.path.getctime))
df_ss = pd.read_excel(max(loadup2, key=os.path.getctime), keep_default_na=False)

In [36]:
SmartSheets = df_ss.copy()
SmartSheets = SmartSheets[['APN_ROW Segment', 'ROE Verified','ROE Type', 'Hold', 'Hold Reason',
                         'Not Eligible','Withdrawal','Phase 1 Complete','Site Assessment', 'USFS Parcel?', 'Debris Crew']]

In [37]:
# df_ss.info(verbose=True)

In [38]:
rdy_SAs = df2.copy()

In [39]:
rdy_SAs = rdy_SAs[(rdy_SAs['ROE Verified'] == 'ROE Verified') & # Roe verified
           (rdy_SAs['ROE Type'] == 'Full Program ROE (debris + trees)') & # ROE types is not just trees
           (rdy_SAs['Hold'].isnull()) & # not on hold 
           (rdy_SAs['Not Eligible'].isnull()) & # Not eilibale = blank
           (rdy_SAs['Withdrawal'].isnull()) & # Withdrawal= blank
           (rdy_SAs['Phase 1 Complete'] == 'Yes') & #Phase one complete
           (rdy_SAs['Site Assessment'].isnull()) & # SA hasnt been done
           (rdy_SAs['USFS Parcel?'].isnull())           
           ]

In [40]:
rdy_SAs = rdy_SAs[['APN_ROW Segment', 'ROE Verified','ROE Type', 'Hold',
                         'Not Eligible','Withdrawal','Phase 1 Complete','Site Assessment', 'USFS Parcel?']]

In [41]:
rdy_SAs = rdy_SAs.add_suffix("_Planning")

In [42]:
SS_merge = rdy_SAs.merge(SmartSheets,
                            left_on='APN_ROW Segment_Planning',
                            right_on='APN_ROW Segment',
                            how='outer')

In [43]:
# SS_merge_RandyFilt = SS_merge.merge(Randy_filt,
#                                left_on='APN_ROW Segment',
#                                right_on='APN_ROW Segment_Randy',
#                                how='outer')

In [44]:
final = SS_merge.merge(DTSC_copy,
                                 left_on='APN_ROW Segment',
                                 right_on='APN_DTSC',
                                 how='outer')

In [45]:
final.set_index('APN_ROW Segment', inplace=True)

In [46]:
final.columns

Index(['APN_ROW Segment_Planning', 'ROE Verified_Planning',
       'ROE Type_Planning', 'Hold_Planning', 'Not Eligible_Planning',
       'Withdrawal_Planning', 'Phase 1 Complete_Planning',
       'Site Assessment_Planning', 'USFS Parcel?_Planning', 'ROE Verified',
       'ROE Type', 'Hold', 'Hold Reason', 'Not Eligible', 'Withdrawal',
       'Phase 1 Complete', 'Site Assessment', 'USFS Parcel?', 'Debris Crew',
       'APN_DTSC', 'County_DTSC', 'Parcel Status:_DTSC', 'Address_DTSC'],
      dtype='object')

In [47]:
# # order columns:
# final = [['Street #', 'Street Name', 'Phase 1 Complete_SS', 'County', # 
#          'APN','County_DTSC', 'Parcel Status:', 'Address_DTSC',
#          'ROE Verified_SS','ROE Type_SS', 'Hold_SS', 'Not Eligible_SS', 'Withdrawal_SS',
#        'Phase 1 Complete_Randy', 'Site Assessment_SS', 'USFS Parcel?_SS',]]

In [48]:
final.to_excel('DTSC and rdy for SA check - Central.xlsx')